In [1]:
import pickle
df = pickle.load(open('Clayton_SG_df_all.pkl', 'br'))
df['long_post'] = df['long_post'].apply(', '.join)
df['long_post'] = df['long_post'].str.replace(',', '')
df.sample(5)

,post_number,long_post,group
5491,39356,husband last chemo treatment week finished che...,Hodgkins Lymphoma
16436,161085,dear husband,Incest Survivors
18733,159440,mothers day todayi hope dealing day better fee...,Infertility
5885,410406,become suddenly overwhelmed lonliness tonight ...,Panic Attack
4581,54470,well hello everyone hoping someone help husban...,Oxycodone


In [2]:
from sklearn.model_selection import train_test_split
X = df.long_post
y = df.group
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40294,)
(10074,)
(40294,)
(10074,)


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(40294, 46697)

In [5]:
# TF-IDF: term-frequency times inverse document frequency
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(40294, 46697)

In [6]:
# Classifier using Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [7]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])

text_clf = text_clf.fit(X_train, y_train)

In [8]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.05787174905697836

In [9]:
# support vector machine classifier
from sklearn.linear_model import SGDClassifier

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, max_iter=5, tol=None, random_state=42)),
])

_ = text_clf_svm.fit(X_train, y_train)

predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.3461385745483423

In [10]:
# Tuning hyperparameters
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),
}

In [11]:
# Grid search with Naive Bayes
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [12]:
%%time
gs_clf = gs_clf.fit(X_train, y_train)
print(gs_clf.best_score_)
gs_clf.best_params_

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


0.24911897552985557
CPU times: user 16.1 s, sys: 3.42 s, total: 19.6 s
Wall time: 54.1 s


In [13]:
# Grid search with SVM classifier
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf-svm__alpha': (1e-2, 1e-3),
}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)

In [14]:
%%time
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
gs_clf_svm.best_params_

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


0.33364768948230505
CPU times: user 3min 4s, sys: 16 s, total: 3min 20s
Wall time: 3min 31s


In [15]:
import nltk
nltk.download('stopwords')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('mnb', MultinomialNB(fit_prior=False)),
])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
%%time
text_mnb_stemmed = text_mnb_stemmed.fit(X_train, y_train)

CPU times: user 24.3 s, sys: 197 ms, total: 24.5 s
Wall time: 24.5 s


In [17]:
predicted_mnb_stemmed = text_mnb_stemmed.predict(X_test)
np.mean(predicted_mnb_stemmed == y_test)

0.08755211435378202

In [18]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [19]:
vectorizers = [
    TfidfVectorizer(stop_words='english',
                    max_features=None),
    CountVectorizer(stop_words='english',
                   max_features=None)
]

classifiers = [
    MultinomialNB(),
    LinearSVC(),
    LogisticRegression(),
    RandomForestClassifier()
]

clf_names = [
         "Naive Bayes",
         "Linear SVC",
         "Logistic Regression",
         "Random Forest",
        ]

vect_names = [
    "TfidfVectorizer",
    "CountVectorizer"
]

clf_params = [
              {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__alpha': (1e-2, 1e-3)},
              {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__C': (np.logspace(-5, 1, 5))},
              {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__C': (np.logspace(-5, 1, 5))},
              {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__max_depth': (1, 2)},
             ]

In [ ]:
%%time
models = []
for classifier, clf_name, params in zip(classifiers, 
                                        clf_names, 
                                        clf_params):
    for vectorizer, vect_name in zip(vectorizers, 
                                     vect_names):
        pipe = Pipeline([
            ('vect', vectorizer),
            ('clf', classifier),
        ])
        gs = GridSearchCV(pipe, 
                          param_grid=params, 
                          n_jobs=-1,
                          scoring='accuracy',
                          cv=5,
                          verbose=10)
        
        gs.fit(df.long_post, df.group)
        score = gs.best_score_
        print(f'''
Classifier: {clf_name}
Vectorizer: {vect_name}
Score: {gs.best_score_:.4f}
Params: {gs.best_params_}
------------------------------
            ''')
        models.append((clf_name, vect_name, gs.best_score_, gs.best_params_))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 1) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.01, vect__ngram_range=(1, 2) .......................
[CV] clf__alpha=0.001, vect__ngram_range=(1, 1) ......................
[CV] clf__alpha=0.001, vect__ngram_range=(1, 1) ......................
[CV] clf__alpha=0.001, vect__ngram_range=(1, 1) ......................
[CV] clf__alpha=0

[Parallel(n_jobs=-1)]: Done   4 out of  20 | elapsed:   10.4s remaining:   41.5s


[CV]  clf__alpha=0.01, vect__ngram_range=(1, 1), score=0.08267361459064035, total=   6.1s
[CV]  clf__alpha=0.001, vect__ngram_range=(1, 1), score=0.25573542556361106, total=   6.4s
[CV]  clf__alpha=0.001, vect__ngram_range=(1, 1), score=0.43161930980237934, total=   6.5s
[CV]  clf__alpha=0.001, vect__ngram_range=(1, 1), score=0.21637251969289062, total=   6.4s


[Parallel(n_jobs=-1)]: Done   7 out of  20 | elapsed:   11.4s remaining:   21.1s


...

[CV]  clf__max_depth=1, vect__ngram_range=(1, 2), score=0.05972542338911715, total=  12.8s
[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.06331727460426703, total=  12.8s


[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:   20.3s remaining:    5.1s


[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.059782608695652176, total=   9.1s
[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.06435594398649319, total=   8.9s
[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.062020141589390766, total=  10.8s
[CV]  clf__max_depth=2, vect__ngram_range=(1, 2), score=0.06162942178575007, total=  11.0s


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   24.5s finished



Classifier: Random Forest
Vectorizer: CountVectorizer
Score: 0.0687
Params: {'clf__max_depth': 2, 'vect__ngram_range': (1, 1)}
------------------------------
            
CPU times: user 2h 45min 36s, sys: 9min 18s, total: 2h 54min 54s
Wall time: 9h 59min 41s


In [22]:
models = sorted(models, key=lambda tup: tup[2])
print('And the winner is...')
print()
print('Classifier:', models[-1][0])
print('Vectorizer:', models[-1][1])
print('Score:', models[-1][2])
print('Params:', models[-1][3])

And the winner is...

Classifier: Linear SVC
Vectorizer: TfidfVectorizer
Score: 0.3417844663278272
Params: {'clf__C': 0.31622776601683794, 'vect__ngram_range': (1, 1)}
